In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os
import mysql.connector
import datetime

In [ ]:
start_time = datetime.datetime.now()
st_time = start_time.strftime('%H:%M')
print("Start Time:", st_time)

In [ ]:
database = input('Enter Database name: ')
category = input('Enter Category Name: ')

In [ ]:
DB = create_engine(f'mysql+pymysql://{username}:{password}@{hostname}/{database}').connect()

In [ ]:
query1 = f'''
select
 
year(date) as year,
month(date) as month,
brand,
subcategory as category,
ROUND(IFNULL(sum(cost_wo_gst)/(sum(cluster_grps*duration)/30),0),2) as cprp_brand_tg,
ROUND(IFNULL(sum(cost_wo_gst)/(sum(max_uni*duration)/30),0),2) as cprp_buying_tg,
ROUND(avg(duration),0) as acd,
count(*) as spot_count
 
from strikesheet
 
where subcategory="{category}" and cat_noncat='CAT'
 
group by 1,2,3,4
order by 1,2,3,4;

'''

In [ ]:
#CPRP_ACD_spot_count query
cprp_acd_query = pd.read_sql(query1,DB)
cprp_acd_query.shape

In [ ]:
query2 = f'''
with d1 as 
(
select
 
year(date) as year,
month(date) as month,
brand,
sum(cost_wo_gst) as spends
from strikesheet
where subcategory='{category}'
and cat_noncat='CAT'
 
group by 1,2,3
order by 1,2,3
),
d2 as 
( 
Select
Year(date) as year,
Month(date) as month,
sum(cost_wo_gst) as spends
From  strikesheet
where subcategory='{category}'
and cat_noncat='CAT'
group by 1,2
order by 1,2
)

select d1.year, d1.month, d1.brand, 
sum(d1.spends) as month_spends, 
d1.spends/d2.spends as category_sos
from d1
join d2 ON d1.year =d2.year AND d1.month = d2.month
group by 1,2,3
order by 1,2,3;

'''

In [ ]:
category_sos = pd.read_sql(query2,DB)
category_sos.shape

In [ ]:
query3 = f'''
with d1 as 
(
select
 
year(date) as year,
month(date) as month,
brand,
sum(pb_grps*duration)/30 as ngrps_pb
from strikesheet
where subcategory='{category}'
and cat_noncat='CAT'
 
group by 1,2,3
order by 1,2,3
),
d2 as 
( 
Select
Year(date) as year,
Month(date) as month,
sum(pb_grps*duration)/30 as ngrps_pb
From  strikesheet
where subcategory='{category}'
and cat_noncat='CAT'
group by 1,2
order by 1,2
)

select d1.year, d1.month, d1.brand, 
sum(d1.ngrps_pb) as month_ngrps, 
d1.ngrps_pb/d2.ngrps_pb as category_sov
from d1
join d2 ON d1.year =d2.year AND d1.month = d2.month
group by 1,2,3
order by 1,2,3;

'''

In [ ]:
category_sov = pd.read_sql(query3,DB)
category_sov.shape

In [ ]:
query4 = f'''
with data as 
(
select
year(date) as year,
month(date) as month,
brand,
if(hour(time) in (20,21,22), 'PT','NON PT') as pt_tag,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}'
and cat_noncat='CAT'
 
group by 1,2,3,4
order by 1,2,3,4
)
select d1.year,
d1.month,
d1.brand,
d2.spends as brand_pt_spends,
d2.ngrps_pb as brand_pt_ngrps,
d2.spends/(d2.spends + d3.spends) as brand_pt_sos,
d2.ngrps_pb/(d2.ngrps_pb + d3.ngrps_pb) as brand_pt_sov

from (select * from data) d1,
(select * from data where pt_tag='PT') d2, 
(select * from data where pt_tag='NON PT') d3
where d1.year=d2.year and d2.year=d3.year and d1.month=d2.month and d2.month=d3.month
and d1.brand=d2.brand and d2.brand=d3.brand
group by 1,2,3
order by 1,2,3
;

'''

In [ ]:
#Brand PT SOS/SOV And NGRPs/Spends
brand_pt = pd.read_sql(query4,DB)
brand_pt.shape

In [ ]:
query5 = f'''
with d1 as 
(
select
 
year(date) as year,
month(date) as month,
brand,
if(hour(time) IN (20,21,22), 'PT', 'NON PT') as pt_tag,
sum(cost_wo_gst) as spends,
sum(pb_grps*duration)/30 as ngrps_pb
from  strikesheet
where subcategory='{category}'
and cat_noncat='CAT'
 
group by 1,2,3,4
order by 1,2,3,4
),
d2 as 
( 
Select
Year(date) as year,
Month(date) as month,
if(hour(time) IN (20,21,22), 'PT', 'NON PT') as pt_tag,
sum(cost_wo_gst) as category_spends,
sum(pb_grps*duration)/30 as ngrps_pb
From  strikesheet
where subcategory='{category}'
and cat_noncat='CAT'
group by 1,2,3
order by 1,2,3
)

select d1.year, d1.month, d1.brand, 
d1.ngrps_pb/d2.ngrps_pb as category_pt_sov,
d1.spends/d2.category_spends as category_pt_sos

from (select * from d1 where pt_tag = 'PT') d1
join (select * from d2 where pt_tag = 'PT') d2 ON d1.year =d2.year AND d1.month = d2.month
group by 1,2,3
order by 1,2,3;

'''

In [ ]:
#Category PT SOV & SOS and brand PT NGRPs & Spends
category_pt = pd.read_sql(query5,DB)
category_pt.shape

In [ ]:
query6 = f'''
with d1 as 
(
select
year(date) as year,
month(date) as month,
brand,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}' and genre = 'Entertainment'
and cat_noncat='CAT'
 
group by 1,2,3
order by 1,2,3
),

d2 as
(
select
year(date) as year,
month(date) as month,
brand,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}'
and genre <> 'entertainment'
and cat_noncat='CAT'
 
group by 1,2,3
order by 1,2,3
)

select d1.year, d1.month, d1.brand,
 d1.spends as brand_ent_spends,
 d1.spends/(d1.spends + d2.spends) as brand_ent_sos,
 d1.ngrps_pb as brand_ent_ngrps,
 d1.ngrps_pb/(d1.ngrps_pb + d2.ngrps_pb) as brand_ent_sov

from d1
join d2
on d1.year = d2.year AND d1.month = d2.month and d1.brand = d2.brand
group by 1,2,3
order by 1,2,3;

'''

In [ ]:
#Entertainment Brand NGrps_Spends_SOS_SOV
entertainment_metrics = pd.read_sql(query6,DB)
entertainment_metrics.shape

In [ ]:
query7 = f'''
with d1 as 
(
select
year(date) as year,
month(date) as month,
brand,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}' and genre = 'News'
and cat_noncat='CAT'
 
group by 1,2,3
order by 1,2,3
),

d2 as
(
select
year(date) as year,
month(date) as month,
brand,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}'
and genre <> 'news'
and cat_noncat='CAT'
 
group by 1,2,3
order by 1,2,3
)

select d1.year, d1.month, d1.brand,
 d1.spends as brand_news_spends,
 d1.spends/(d1.spends + d2.spends) as brand_news_sos,
 d1.ngrps_pb as brand_news_ngrps,
 d1.ngrps_pb/(d1.ngrps_pb + d2.ngrps_pb) as brand_news_sov

from d1
join d2
on d1.year = d2.year AND d1.month = d2.month and d1.brand = d2.brand
group by 1,2,3
order by 1,2,3;

'''

In [ ]:
#News Brand NGrps_Spends_SOS_SOV
news_metrics = pd.read_sql(query7,DB)
news_metrics.shape

In [ ]:
query8 = f'''
with data as 
(
select
year(date) as year,
month(date) as month,
brand,
CASE WHEN channel IN ("ARY DIGITAL","ARY NEWS","GREEN ENTERTAINMENT","GEO ENTERTAINMENT","GEO NEWS","HUM TV")
THEN "Tier 1"
WHEN channel IN ("A SPORTS","TEN SPORTS","SAMAA","DUNYA NEWS","PTV SPORTS","BOL ENTERTAINMENT","24 NEWS","AAN TV","BOL News","CARTOON NETWORK","DAWN NEWS","EXPRESS NEWS","EXPRESS ENTERTAINMENT","GEO KAHANI","HUM NEWS","KID ZONE PLUS","NEO TV","URDU 1","KIDZONE TV","SUNO TV","PTV HOME","TALON NEWS","MASALA TV","A PLUS","FAST SPORTS","HOORA TV","DISCOVER PAKISTAN")
THEN "Tier 2"
ELSE "Tier 3"
END as Tiers,

sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}'
and cat_noncat='CAT'
 
group by 1,2,3,4
order by 1,2,3,4
)

select d1.year,
d1.month,
d1.brand,
d2.ngrps_pb as brand_tier1_ngrps,
d2.ngrps_pb/(d2.ngrps_pb + d3.ngrps_pb + d4.ngrps_pb) as brand_tier1_sov,
d2.spends as brand_tier1_spends,
d2.spends/(d2.spends + d3.spends + d4.spends) as brand_tier1_sos


from (select * from data) d1,
(select * from data where Tiers ='Tier 1') d2, 
(select * from data where Tiers ='Tier 2') d3,
(select * from data where Tiers ='Tier 3') d4
where d1.year=d2.year and d2.year=d3.year and d1.month=d2.month and d2.month=d3.month
and d1.brand=d2.brand and d2.brand=d3.brand
group by 1,2,3
order by 1,2,3
;

'''

In [ ]:
#Tier 1 Brand SOS_SOV and NGRPs_Spends
tier1_metrics = pd.read_sql(query8,DB)
tier1_metrics.shape

In [ ]:
query9 = f'''
with data1 as 
(
select
year(date) as year,
month(date) as month,
brand,
CASE WHEN channel IN ("ARY DIGITAL","ARY NEWS","GREEN ENTERTAINMENT","GEO ENTERTAINMENT","GEO NEWS","HUM TV")
THEN "Tier 1"
WHEN channel IN ("A SPORTS","TEN SPORTS","SAMAA","DUNYA NEWS","PTV SPORTS","BOL ENTERTAINMENT","24 NEWS","AAN TV","BOL News","CARTOON NETWORK","DAWN NEWS","EXPRESS NEWS","EXPRESS ENTERTAINMENT","GEO KAHANI","HUM NEWS","KID ZONE PLUS","NEO TV","URDU 1","KIDZONE TV","SUNO TV","PTV HOME","TALON NEWS","MASALA TV","A PLUS","FAST SPORTS","HOORA TV","DISCOVER PAKISTAN")
THEN "Tier 2"
ELSE "Tier 3"
END as Tiers,

sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends

from  strikesheet

where subcategory='{category}' and genre = 'Entertainment'
and cat_noncat='CAT'
 
group by 1,2,3,4
order by 1,2,3,4
),

data2 as
(
select
year(date) as year,
month(date) as month,
brand,
CASE WHEN channel IN ("ARY DIGITAL","ARY NEWS","GREEN ENTERTAINMENT","GEO ENTERTAINMENT","GEO NEWS","HUM TV")
THEN "Tier 1"
WHEN channel IN ("A SPORTS","TEN SPORTS","SAMAA","DUNYA NEWS","PTV SPORTS","BOL ENTERTAINMENT","24 NEWS","AAN TV","BOL News","CARTOON NETWORK","DAWN NEWS","EXPRESS NEWS","EXPRESS ENTERTAINMENT","GEO KAHANI","HUM NEWS","KID ZONE PLUS","NEO TV","URDU 1","KIDZONE TV","SUNO TV","PTV HOME","TALON NEWS","MASALA TV","A PLUS","FAST SPORTS","HOORA TV","DISCOVER PAKISTAN")
THEN "Tier 2"
ELSE "Tier 3"
END as Tiers,

sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}'
and genre <> 'entertainment'
and cat_noncat='CAT'
 
group by 1,2,3,4
order by 1,2,3,4
)

select d1.year, d1.month, d1.brand,
 d1.spends as brand_tier1_ent_spends,
 d1.spends/(d1.spends + d2.spends) as brand_tier1_ent_sos,
 d1.ngrps_pb as brand_tier1_ent_ngrps,
 d1.ngrps_pb/(d1.ngrps_pb + d2.ngrps_pb) as brand_tier1_ent_sov

from (select * from data1 where Tiers = 'Tier 1') d1 
join (select * from data2 where Tiers = 'Tier 1') d2
on d1.year = d2.year AND d1.month = d2.month and d1.brand = d2.brand
group by 1,2,3
order by 1,2,3;

'''

In [ ]:
#Tier 1 Ent brand SOS_SOV and NGRPs_Spends
tier1_ent_metrics = pd.read_sql(query9,DB)
tier1_ent_metrics.shape

In [ ]:
query10 = f'''
with data1 as 
(
select
year(date) as year,
month(date) as month,
brand,
CASE WHEN channel IN ("ARY DIGITAL","ARY NEWS","GREEN ENTERTAINMENT","GEO ENTERTAINMENT","GEO NEWS","HUM TV")
THEN "Tier 1"
WHEN channel IN ("A SPORTS","TEN SPORTS","SAMAA","DUNYA NEWS","PTV SPORTS","BOL ENTERTAINMENT","24 NEWS","AAN TV","BOL News","CARTOON NETWORK","DAWN NEWS","EXPRESS NEWS","EXPRESS ENTERTAINMENT","GEO KAHANI","HUM NEWS","KID ZONE PLUS","NEO TV","URDU 1","KIDZONE TV","SUNO TV","PTV HOME","TALON NEWS","MASALA TV","A PLUS","FAST SPORTS","HOORA TV","DISCOVER PAKISTAN")
THEN "Tier 2"
ELSE "Tier 3"
END as Tiers,

sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends

from  strikesheet

where subcategory='{category}' and genre = 'news'
and cat_noncat='CAT'
 
group by 1,2,3,4
order by 1,2,3,4
),

data2 as
(
select
year(date) as year,
month(date) as month,
brand,
CASE WHEN channel IN ("ARY DIGITAL","ARY NEWS","GREEN ENTERTAINMENT","GEO ENTERTAINMENT","GEO NEWS","HUM TV")
THEN "Tier 1"
WHEN channel IN ("A SPORTS","TEN SPORTS","SAMAA","DUNYA NEWS","PTV SPORTS","BOL ENTERTAINMENT","24 NEWS","AAN TV","BOL News","CARTOON NETWORK","DAWN NEWS","EXPRESS NEWS","EXPRESS ENTERTAINMENT","GEO KAHANI","HUM NEWS","KID ZONE PLUS","NEO TV","URDU 1","KIDZONE TV","SUNO TV","PTV HOME","TALON NEWS","MASALA TV","A PLUS","FAST SPORTS","HOORA TV","DISCOVER PAKISTAN")
THEN "Tier 2"
ELSE "Tier 3"
END as Tiers,

sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}'
and genre <> 'news'
and cat_noncat='CAT'
 
group by 1,2,3,4
order by 1,2,3,4
)

select d1.year, d1.month, d1.brand,
 d1.spends as brand_tier1_news_spends,
 d1.spends/(d1.spends + d2.spends) as brand_tier1_news_sos,
 d1.ngrps_pb as brand_tier1_news_ngrps,
 d1.ngrps_pb/(d1.ngrps_pb + d2.ngrps_pb) as brand_tier1_news_sov

from (select * from data1 where Tiers = 'Tier 1') d1 
join (select * from data2 where Tiers = 'Tier 1') d2
on d1.year = d2.year AND d1.month = d2.month and d1.brand = d2.brand
group by 1,2,3
order by 1,2,3;

'''

In [ ]:
#Tier 1 News brand SOS_SOV and NGRPs_Spends
tier1_news_metrics = pd.read_sql(query10,DB)
tier1_news_metrics.shape

In [ ]:
query11 = f'''
with data as 
(
select
year(date) as year,
month(date) as month,
brand,
CASE WHEN channel IN ("ARY DIGITAL","ARY NEWS","GREEN ENTERTAINMENT","GEO ENTERTAINMENT","GEO NEWS","HUM TV")
THEN "Tier 1"
WHEN channel IN ("A SPORTS","TEN SPORTS","SAMAA","DUNYA NEWS","PTV SPORTS","BOL ENTERTAINMENT","24 NEWS","AAN TV","BOL News","CARTOON NETWORK","DAWN NEWS","EXPRESS NEWS","EXPRESS ENTERTAINMENT","GEO KAHANI","HUM NEWS","KID ZONE PLUS","NEO TV","URDU 1","KIDZONE TV","SUNO TV","PTV HOME","TALON NEWS","MASALA TV","A PLUS","FAST SPORTS","HOORA TV","DISCOVER PAKISTAN")
THEN "Tier 2"
ELSE "Tier 3"
END as Tiers,
if(hour(time) in (20,21,22), 'PT','NON PT') as pt_tag,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}'
and cat_noncat='CAT'
 
group by 1,2,3,4,5
order by 1,2,3,4,5
)


SELECT d1.year, d1.month, d1.brand, 
d2.ngrps_pb as brand_tier1_pt_ngrps,
d2.spends as brand_tier1_pt_spends,
d2.ngrps_pb/(d2.ngrps_pb + d3.ngrps_pb) as brand_tier1_pt_sov,
d2.spends/(d2.spends + d3.spends) as brand_tier1_pt_sos

from (select * from data) d1,
(select * from data where Tiers = 'Tier 1' and pt_tag = 'PT') d2,
(select * from data where Tiers = 'Tier 1') d3
where d1.year = d2.year and d2.year = d3.year and d1.month = d2.month and d2.month = d3.month and d1.brand = d2.brand and d2.brand = d3.brand

group by 1,2,3
order by 1,2,3;

'''

In [ ]:
#Tier 1 PT Brand SOV_SOS and NGRPs_Spends
tier1_pt_metrics = pd.read_sql(query11,DB)
tier1_pt_metrics.shape

In [ ]:
query12 = f'''
with data as 
(
select
year(date) as year,
month(date) as month,
brand,
CASE WHEN channel IN ("ARY DIGITAL","ARY NEWS","GREEN ENTERTAINMENT","GEO ENTERTAINMENT","GEO NEWS","HUM TV")
THEN "Tier 1"
WHEN channel IN ("A SPORTS","TEN SPORTS","SAMAA","DUNYA NEWS","PTV SPORTS","BOL ENTERTAINMENT","24 NEWS","AAN TV","BOL News","CARTOON NETWORK","DAWN NEWS","EXPRESS NEWS","EXPRESS ENTERTAINMENT","GEO KAHANI","HUM NEWS","KID ZONE PLUS","NEO TV","URDU 1","KIDZONE TV","SUNO TV","PTV HOME","TALON NEWS","MASALA TV","A PLUS","FAST SPORTS","HOORA TV","DISCOVER PAKISTAN")
THEN "Tier 2"
ELSE "Tier 3"
END as Tiers,
if(hour(time) in (20,21,22), 'PT','NON PT') as pt_tag,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where genre = 'entertainment' and
subcategory='{category}'
and cat_noncat='CAT'
 
group by 1,2,3,4,5
order by 1,2,3,4,5
)


SELECT d1.year, d1.month, d1.brand, 
d2.ngrps_pb as brand_tier1_ent_pt_ngrps,
d2.spends as brand_tier1_ent_pt_spends,
d2.ngrps_pb/(d2.ngrps_pb + d3.ngrps_pb) as brand_tier1_ent_pt_sov,
d2.spends/(d2.spends + d3.spends) as brand_tier1_ent_pt_sos

from (select * from data) d1,
(select * from data where Tiers = 'Tier 1' and pt_tag = 'PT') d2,
(select * from data where Tiers = 'Tier 1') d3
where d1.year = d2.year and d2.year = d3.year and d1.month = d2.month and d2.month = d3.month and d1.brand = d2.brand and d2.brand = d3.brand

group by 1,2,3
order by 1,2,3;

'''

In [ ]:
#Tier 1 PT Ent Brand SOV_SOS and NGRPs_Spends
tier1_pt_ent_metrics = pd.read_sql(query12,DB)
tier1_pt_ent_metrics.shape

In [ ]:
query13 = f'''
with data as 
(
select
year(date) as year,
month(date) as month,
brand,
CASE WHEN channel IN ("ARY DIGITAL","ARY NEWS","GREEN ENTERTAINMENT","GEO ENTERTAINMENT","GEO NEWS","HUM TV")
THEN "Tier 1"
WHEN channel IN ("A SPORTS","TEN SPORTS","SAMAA","DUNYA NEWS","PTV SPORTS","BOL ENTERTAINMENT","24 NEWS","AAN TV","BOL News","CARTOON NETWORK","DAWN NEWS","EXPRESS NEWS","EXPRESS ENTERTAINMENT","GEO KAHANI","HUM NEWS","KID ZONE PLUS","NEO TV","URDU 1","KIDZONE TV","SUNO TV","PTV HOME","TALON NEWS","MASALA TV","A PLUS","FAST SPORTS","HOORA TV","DISCOVER PAKISTAN")
THEN "Tier 2"
ELSE "Tier 3"
END as Tiers,
if(hour(time) in (20,21,22), 'PT','NON PT') as pt_tag,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where genre = 'news' and
subcategory='{category}'
and cat_noncat='CAT'
 
group by 1,2,3,4,5
order by 1,2,3,4,5
)


SELECT d1.year, d1.month, d1.brand, 
d2.ngrps_pb as brand_tier1_news_pt_ngrps,
d2.spends as brand_tier1_news_pt_spends,
d2.ngrps_pb/(d2.ngrps_pb + d3.ngrps_pb) as brand_tier1_news_pt_sov,
d2.spends/(d2.spends + d3.spends) as brand_tier1_news_pt_sos

from (select * from data) d1,
(select * from data where Tiers = 'Tier 1' and pt_tag = 'PT') d2,
(select * from data where Tiers = 'Tier 1') d3
where d1.year = d2.year and d2.year = d3.year and d1.month = d2.month and d2.month = d3.month and d1.brand = d2.brand and d2.brand = d3.brand

group by 1,2,3
order by 1,2,3;

'''

In [ ]:
#Tier 1 PT News Brand SOV_SOS and NGRPs_Spends
tier1_pt_news_metrics = pd.read_sql(query13,DB)
tier1_pt_news_metrics.shape

In [ ]:
query14 = f'''
with data as 
(
select
year(date) as year,
month(date) as month,
brand,
if(hour(time) in (18,19,20,21,22,23), 'EPT','NON EPT') as ept_tag,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}'
and cat_noncat='CAT'
 
group by 1,2,3,4
order by 1,2,3,4
)
select d1.year,
d1.month,
d1.brand,
d2.ngrps_pb/(d2.ngrps_pb + d3.ngrps_pb) as brand_ept_sov, 
d2.spends/(d2.spends + d3.spends) as brand_ept_sos,
d2.ngrps_pb as brand_ept_ngrps,
d2.spends as brand_ept_spends

from (select * from data) d1,
(select * from data where ept_tag='EPT') d2, 
(select * from data where ept_tag='NON EPT') d3
where d1.year=d2.year and d2.year=d3.year and d1.month=d2.month and d2.month=d3.month
and d1.brand=d2.brand and d2.brand=d3.brand
group by 1,2,3
order by 1,2,3
;
'''

In [ ]:
#Extended Prime Time SOV_SOS and NGRPs_Spends
ept_metrics = pd.read_sql(query14,DB)
ept_metrics.shape

In [ ]:
query15 = f'''
with data as 
(
select
year(date) as year,
month(date) as month,
brand,
if(hour(time) in (7,8,9,10,11), 'Morning','NON Morning') as Morning_tag,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}'
and cat_noncat='CAT'
 
group by 1,2,3,4
order by 1,2,3,4
)
select d1.year,
d1.month,
d1.brand,
d2.ngrps_pb as brand_morning_ngrps,
d2.spends as brand_morning_spends,
d2.spends/(d2.spends + d3.spends) as brand_morning_sos,
d2.ngrps_pb/(d2.ngrps_pb + d3.ngrps_pb) as brand_morning_sov

from (select * from data) d1,
(select * from data where morning_tag='Morning') d2, 
(select * from data where morning_tag='NON Morning') d3
where d1.year=d2.year and d2.year=d3.year and d1.month=d2.month and d2.month=d3.month
and d1.brand=d2.brand and d2.brand=d3.brand
group by 1,2,3
order by 1,2,3
;
'''

In [ ]:
#Morning Time SOV_SOS and NGRPs_Spends
morning_metrics = pd.read_sql(query15,DB)
morning_metrics.shape

In [ ]:
query16 = f'''
with data as 
(
select
year(date) as year,
month(date) as month,
brand,
if(hour(time) in (12,13), 'Afternoon','NON Afternoon') as Afternoon_tag,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}'
and cat_noncat='CAT'
 
group by 1,2,3,4
order by 1,2,3,4
)
select d1.year,
d1.month,
d1.brand,
d2.ngrps_pb as brand_afternoon_ngrps,
d2.spends as brand_afternoon_spends,
d2.spends/(d2.spends + d3.spends) as brand_afternoon_sos,
d2.ngrps_pb/(d2.ngrps_pb + d3.ngrps_pb) as brand_afternoon_sov

from (select * from data) d1,
(select * from data where Afternoon_tag='Afternoon') d2, 
(select * from data where Afternoon_tag='NON Afternoon') d3
where d1.year=d2.year and d2.year=d3.year and d1.month=d2.month and d2.month=d3.month
and d1.brand=d2.brand and d2.brand=d3.brand
group by 1,2,3
order by 1,2,3
;
'''

In [ ]:
#Afternoon Time SOV_SOS and NGRPs_Spends
Afternoon_metrics = pd.read_sql(query16,DB)
Afternoon_metrics.shape

In [ ]:
query17 = f'''
with data as 
(
select
year(date) as year,
month(date) as month,
brand,
if(hour(time) in (14,15,16,17), 'Matinee','NON Matinee') as Matinee_tag,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}'
and cat_noncat='CAT'
 
group by 1,2,3,4
order by 1,2,3,4
)
select d1.year,
d1.month,
d1.brand,
d2.ngrps_pb as brand_matinee_ngrps,
d2.spends as brand_matinee_spends,
d2.spends/(d2.spends + d3.spends) as brand_matinee_sos,
d2.ngrps_pb/(d2.ngrps_pb + d3.ngrps_pb) as brand_matinee_sov

from (select * from data) d1,
(select * from data where Matinee_tag='Matinee') d2, 
(select * from data where Matinee_tag='NON Matinee') d3
where d1.year=d2.year and d2.year=d3.year and d1.month=d2.month and d2.month=d3.month
and d1.brand=d2.brand and d2.brand=d3.brand
group by 1,2,3
order by 1,2,3
;
'''

In [ ]:
#Matinee Time SOV_SOS and NGRPs_Spends
Matinee_metrics = pd.read_sql(query17,DB)
Matinee_metrics.shape

In [ ]:
query18 = f'''
with d1 as 
(
select
year(date) as year,
month(date) as month,
brand,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}' and genre = 'Sports'
and cat_noncat='CAT'
 
group by 1,2,3
order by 1,2,3
),

d2 as
(
select
year(date) as year,
month(date) as month,
brand,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}'
and genre <> 'sports'
and cat_noncat='CAT'
 
group by 1,2,3
order by 1,2,3
)

select d1.year, d1.month, d1.brand,
 d1.spends as brand_sports_spends,
 d1.spends/(d1.spends + d2.spends) as brand_sports_sos,
 d1.ngrps_pb as brand_sports_ngrps,
 d1.ngrps_pb/(d1.ngrps_pb + d2.ngrps_pb) as brand_sports_sov

from d1
join d2
on d1.year = d2.year AND d1.month = d2.month and d1.brand = d2.brand
group by 1,2,3
order by 1,2,3;

'''

In [ ]:
#Sports SOV_SOS and NGRPs_Spends
Brand_Sports = pd.read_sql(query18,DB)
Brand_Sports.shape

In [ ]:
query19 = f'''
with data as 

(
select
year(date) as year,
month(date) as month,
brand,
if(hour(time) in (20,21,22), 'PT','NON PT') as pt_tag,
genre,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}'
and cat_noncat='CAT'
group by 1,2,3,4,5
order by 1,2,3,4,5
)

SELECT d1.year, d1.month, d1.brand, 
d2.ngrps_pb as brand_pt_ent_ngrps,
d2.spends as brand_pt_ent_spends,
d2.ngrps_pb/sum(d1.ngrps_pb + d2.ngrps_pb) as brand_pt_ent_sov,
d2.spends/sum(d1.spends + d2.spends) as brand_pt_ent_sos
from
(select * from data where genre <> 'Entertainment' and pt_tag = 'PT') d1,
(select * from data where genre = 'Entertainment' and pt_tag = 'PT') d2
where d1.year = d2.year and d1.month = d2.month and d1.brand = d2.brand
group by 1,2,3
order by 1,2,3;
'''

In [ ]:
#PT Ent SOV_SOS and NGRPs_Spends
Brand_PT_Ent = pd.read_sql(query19,DB)
Brand_PT_Ent.shape

In [ ]:
query20 = f'''
with data as 

(
select
year(date) as year,
month(date) as month,
brand,
if(hour(time) in (20,21,22), 'PT','NON PT') as pt_tag,
genre,
sum(pb_grps*duration)/30 as ngrps_pb,
sum(cost_wo_gst) as spends
from  strikesheet
where subcategory='{category}'
and cat_noncat='CAT'
group by 1,2,3,4,5
order by 1,2,3,4,5
)

SELECT d1.year, d1.month, d1.brand, 
d2.ngrps_pb as brand_pt_news_ngrps,
d2.spends as brand_pt_news_spends,
d2.ngrps_pb/sum(d1.ngrps_pb + d2.ngrps_pb) as brand_pt_news_sov,
d2.spends/sum(d1.spends + d2.spends) as brand_pt_news_sos
from
(select * from data where genre <> 'News' and pt_tag = 'PT') d1,
(select * from data where genre = 'News' and pt_tag = 'PT') d2
where d1.year = d2.year and d1.month = d2.month and d1.brand = d2.brand
group by 1,2,3
order by 1,2,3;
'''

In [ ]:
#PT News SOV_SOS and NGRPs_Spends
Brand_PT_News = pd.read_sql(query20,DB)
Brand_PT_News.shape

In [ ]:
merge1 = cprp_acd_query.merge(category_sos, on = ['year','month','brand'], how = 'left')

In [ ]:
merge_2 = merge1.merge(category_sov, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_3 = merge_2.merge(brand_pt, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_4 = merge_3.merge(entertainment_metrics, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_5 = merge_4.merge(category_pt, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_6 = merge_5.merge(news_metrics, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_7 = merge_6.merge(tier1_metrics, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_8 = merge_7.merge(tier1_ent_metrics, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_9 = merge_8.merge(tier1_news_metrics, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_10 = merge_9.merge(tier1_pt_metrics, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_11 = merge_10.merge(tier1_pt_ent_metrics, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_12 = merge_11.merge(tier1_pt_news_metrics, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_13 = merge_12.merge(ept_metrics, on= ['year','month','brand'], how = 'left')


In [ ]:
merge_14 = merge_13.merge(morning_metrics, on= ['year','month','brand'], how = 'left')


In [ ]:
merge_15 = merge_14.merge(Matinee_metrics, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_16 = merge_15.merge(Brand_Sports, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_17 = merge_16.merge(Brand_PT_Ent, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_18 = merge_17.merge(Brand_PT_News, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_19 = merge_18.merge(Afternoon_metrics, on= ['year','month','brand'], how = 'left')

In [ ]:
merge_19.head()

In [ ]:
columns_list = merge_19.columns.tolist()

In [ ]:
columns_list

In [ ]:
final_data = merge_19.loc[:,['year', 'month', 'category', 'brand', 'acd', 'cprp_brand_tg', 'cprp_buying_tg', 'month_spends', 'month_ngrps', 'category_sos', 'category_sov', 'category_pt_sos', 'category_pt_sov', 'brand_pt_spends', 'brand_pt_ngrps', 'brand_pt_sos', 'brand_pt_sov', 'brand_morning_spends', 'brand_morning_ngrps', 'brand_morning_sos', 'brand_morning_sov', 'brand_afternoon_spends', 'brand_afternoon_ngrps', 'brand_afternoon_sos', 'brand_afternoon_sov', 'brand_matinee_spends', 'brand_matinee_ngrps', 'brand_matinee_sos', 'brand_matinee_sov', 'brand_ept_spends', 'brand_ept_ngrps', 'brand_ept_sos', 'brand_ept_sov', 'brand_ent_spends', 'brand_ent_ngrps', 'brand_ent_sos', 'brand_ent_sov', 'brand_news_spends', 'brand_news_ngrps', 'brand_news_sos', 'brand_news_sov', 'brand_sports_spends', 'brand_sports_ngrps', 'brand_sports_sos', 'brand_sports_sov', 'brand_tier1_spends', 'brand_tier1_ngrps', 'brand_tier1_sos', 'brand_tier1_sov', 'brand_tier1_ent_spends', 'brand_tier1_ent_ngrps', 'brand_tier1_ent_sos', 'brand_tier1_ent_sov', 'brand_tier1_news_spends', 'brand_tier1_news_ngrps', 'brand_tier1_news_sos', 'brand_tier1_news_sov', 'brand_pt_ent_spends', 'brand_pt_ent_ngrps', 'brand_pt_ent_sov', 'brand_pt_ent_sos', 'brand_pt_news_spends', 'brand_pt_news_ngrps', 'brand_pt_news_sov', 'brand_pt_news_sos', 'brand_tier1_pt_spends', 'brand_tier1_pt_ngrps', 'brand_tier1_pt_sos', 'brand_tier1_pt_sov', 'brand_tier1_ent_pt_spends', 'brand_tier1_ent_pt_ngrps', 'brand_tier1_ent_pt_sos', 'brand_tier1_ent_pt_sov', 'brand_tier1_news_pt_spends', 'brand_tier1_news_pt_ngrps', 'brand_tier1_news_pt_sos', 'brand_tier1_news_pt_sov']]

In [ ]:
final_data_filled_nulls = final_data.fillna(0)

In [ ]:
final_data_filled_nulls['key'] = final_data_filled_nulls['year'].astype(str) + final_data_filled_nulls['month'].astype(str) + final_data_filled_nulls['brand']

In [ ]:
final_mastersheet_data =final_data_filled_nulls.loc[:,['key', 'year', 'month', 'category', 'brand', 'acd', 'cprp_brand_tg', 'cprp_buying_tg', 'month_spends', 'month_ngrps', 'category_sos', 'category_sov', 'category_pt_sos', 'category_pt_sov', 'brand_pt_spends', 'brand_pt_ngrps', 'brand_pt_sos', 'brand_pt_sov', 'brand_morning_spends', 'brand_morning_ngrps', 'brand_morning_sos', 'brand_morning_sov', 'brand_afternoon_spends', 'brand_afternoon_ngrps', 'brand_afternoon_sos', 'brand_afternoon_sov', 'brand_matinee_spends', 'brand_matinee_ngrps', 'brand_matinee_sos', 'brand_matinee_sov', 'brand_ept_spends', 'brand_ept_ngrps', 'brand_ept_sos', 'brand_ept_sov', 'brand_ent_spends', 'brand_ent_ngrps', 'brand_ent_sos', 'brand_ent_sov', 'brand_news_spends', 'brand_news_ngrps', 'brand_news_sos', 'brand_news_sov', 'brand_sports_spends', 'brand_sports_ngrps', 'brand_sports_sos', 'brand_sports_sov', 'brand_tier1_spends', 'brand_tier1_ngrps', 'brand_tier1_sos', 'brand_tier1_sov', 'brand_tier1_ent_spends', 'brand_tier1_ent_ngrps', 'brand_tier1_ent_sos', 'brand_tier1_ent_sov', 'brand_tier1_news_spends', 'brand_tier1_news_ngrps', 'brand_tier1_news_sos', 'brand_tier1_news_sov', 'brand_pt_ent_spends', 'brand_pt_ent_ngrps', 'brand_pt_ent_sov', 'brand_pt_ent_sos', 'brand_pt_news_spends', 'brand_pt_news_ngrps', 'brand_pt_news_sov', 'brand_pt_news_sos', 'brand_tier1_pt_spends', 'brand_tier1_pt_ngrps', 'brand_tier1_pt_sos', 'brand_tier1_pt_sov', 'brand_tier1_ent_pt_spends', 'brand_tier1_ent_pt_ngrps', 'brand_tier1_ent_pt_sos', 'brand_tier1_ent_pt_sov', 'brand_tier1_news_pt_spends', 'brand_tier1_news_pt_ngrps', 'brand_tier1_news_pt_sos', 'brand_tier1_news_pt_sov']]
final_mastersheet_data

In [ ]:
final_mastersheet_data.to_csv(f'categories_strikesheets/{database}_{category}.csv')

In [ ]:
end_time = datetime.datetime.now()
e_time = end_time.strftime('%H:%M')
print("End Time:", e_time)